# Labelme转YOLO-批量

同济子豪兄 2023-4-16

## 导入工具包

In [1]:
import os
import json
import shutil
import numpy as np
from tqdm import tqdm

## 删除系统自动生成的多余文件

### 查看待删除的多余文件

In [1]:
!find . -iname '__MACOSX'

In [2]:
!find . -iname '.DS_Store'

In [3]:
!find . -iname '.ipynb_checkpoints'

./.ipynb_checkpoints


### 删除多余文件

In [4]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

In [5]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

In [6]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

### 验证多余文件已删除

In [7]:
!find . -iname '__MACOSX'

In [8]:
!find . -iname '.DS_Store'

In [9]:
!find . -iname '.ipynb_checkpoints'

## 指定数据集信息

In [11]:
Dataset_root = 'Ruler_15_Dataset'

In [12]:
classes = {
    'Angle':0,
    'Ruler':1,
    'Triangle_30':2,
    'Triangle_45':3
}

## 生成`classes.txt`文件

In [13]:
os.chdir(Dataset_root)

In [14]:
with open('classes.txt', 'w', encoding='utf-8') as f:
    for each in list(classes.keys()):
        f.write(each + '\n')

In [15]:
os.mkdir('labels')
os.mkdir('labels/train')
os.mkdir('labels/val')

## 函数-处理单个labelme标注json文件

In [16]:
def process_single_json(labelme_path, save_folder='../../labels/train'):
    
    # 载入 labelme格式的 json 标注文件
    with open(labelme_path, 'r', encoding='utf-8') as f:
        labelme = json.load(f)
        
    img_width = labelme['imageWidth']   # 图像宽度
    img_height = labelme['imageHeight'] # 图像高度
    
    # 生成 YOLO 格式的 txt 文件
    suffix = labelme_path.split('.')[-2]
    yolo_txt_path = suffix + '.txt'
    
    with open(yolo_txt_path, 'w', encoding='utf-8') as f:
        for each_ann in labelme['shapes']: # 遍历每个框

            if each_ann['shape_type'] == 'rectangle': # 筛选出框

                # 获取类别 ID
                bbox_class_id = classes[each_ann['label']]

                # 左上角和右下角的 XY 像素坐标
                bbox_top_left_x = int(min(each_ann['points'][0][0], each_ann['points'][1][0]))
                bbox_bottom_right_x = int(max(each_ann['points'][0][0], each_ann['points'][1][0]))
                bbox_top_left_y = int(min(each_ann['points'][0][1], each_ann['points'][1][1]))
                bbox_bottom_right_y = int(max(each_ann['points'][0][1], each_ann['points'][1][1]))

                # 框中心点的 XY 像素坐标
                bbox_center_x = int((bbox_top_left_x + bbox_bottom_right_x) / 2)
                bbox_center_y = int((bbox_top_left_y + bbox_bottom_right_y) / 2)

                # 框宽度
                bbox_width = bbox_bottom_right_x - bbox_top_left_x

                # 框高度
                bbox_height = bbox_bottom_right_y - bbox_top_left_y

                # 框中心点归一化坐标
                bbox_center_x_norm = bbox_center_x / img_width
                bbox_center_y_norm = bbox_center_y / img_height

                # 框归一化宽度
                bbox_width_norm = bbox_width / img_width
                # 框归一化高度
                bbox_height_norm = bbox_height / img_height

                # 生成 YOLO 格式的一行标注，指定保留小数点后几位
                bbox_yolo_str = '{} {:.4f} {:.4f} {:.4f} {:.4f}'.format(bbox_class_id, bbox_center_x_norm, bbox_center_y_norm, bbox_width_norm, bbox_height_norm)
                # 写入 txt 文件中
                f.write(bbox_yolo_str + '\n')

    shutil.move(yolo_txt_path, save_folder)
    print('{} --> {} 转换完成'.format(labelme_path, yolo_txt_path))

## 转换训练集标注文件至`labels/train`目录

In [17]:
os.chdir('labelme_jsons/train')

In [18]:
save_folder = '../../labels/train'
for labelme_path in os.listdir():
    process_single_json(labelme_path, save_folder=save_folder)
print('YOLO格式的txt标注文件已保存至 ', save_folder)

11.json --> 11.txt 转换完成
12.json --> 12.txt 转换完成
5.json --> 5.txt 转换完成
1.json --> 1.txt 转换完成
6.json --> 6.txt 转换完成
9.json --> 9.txt 转换完成
7.json --> 7.txt 转换完成
8.json --> 8.txt 转换完成
4.json --> 4.txt 转换完成
15.json --> 15.txt 转换完成
10.json --> 10.txt 转换完成
14.json --> 14.txt 转换完成
YOLO格式的txt标注文件已保存至  ../../labels/train


In [19]:
os.chdir('../../')

## 转换测试集标注文件至`labels/val`目录

In [20]:
os.chdir('labelme_jsons/val')

In [21]:
save_folder = '../../labels/val'
for labelme_path in os.listdir():
    process_single_json(labelme_path, save_folder=save_folder)
print('YOLO格式的txt标注文件已保存至 ', save_folder)

3.json --> 3.txt 转换完成
13.json --> 13.txt 转换完成
2.json --> 2.txt 转换完成
YOLO格式的txt标注文件已保存至  ../../labels/val


In [22]:
os.chdir('../../')

## 删除labelme格式的标注文件

In [23]:
!rm -rf labelme_jsons

## 删除系统自动生成的多余文件

### 查看待删除的多余文件

In [33]:
!find . -iname '__MACOSX'

In [34]:
!find . -iname '.DS_Store'

In [35]:
!find . -iname '.ipynb_checkpoints'

### 删除多余文件

In [36]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

In [37]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

In [38]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

### 验证多余文件已删除

In [39]:
!find . -iname '__MACOSX'

In [40]:
!find . -iname '.DS_Store'

In [41]:
!find . -iname '.ipynb_checkpoints'

## 得到完整的YOLO格式数据集